In [88]:
import json
import pandas as pd
import numpy as np
import mysql.connector
import os

In [89]:
# Getting the users data from database_init.json (add it to .gitignore)
with open('database_init.json') as file:
    db_config = json.load(file)
#creating NBA_DB if it is not already exists.
db_name = "NBA_DB"
try:
    temp_config = db_config.copy()
    cnxn = mysql.connector.connect(**temp_config)
    cursor = cnxn.cursor()

    cursor.execute(f"CREATE DATABASE IF NOT EXISTS {db_name} COLLATE utf8mb4_unicode_ci")
    print(f"Database '{db_name}' created or already exists.")
    cnxn.database = db_name


except mysql.connector.Error as err:
    print(f"Error!!: {err}")
finally:
    if 'cnxn' in locals() and cnxn.is_connected():
        cursor.close()
        cnxn.close()
        print("Connection Closed.")

Database 'NBA_DB' created or already exists.
Connection Closed.


In [90]:
players = pd.DataFrame()
#adding season year to columns and extracting player_id
lst = []
for i in {2019, 2020, 2021, 2022, 2023, 2024, 2025}:
    path = os.path.join('top 50 player of mjt list', f'nba_top50_players_{i}.csv')
    tmp = pd.read_csv(path)
    tmp['Season'] = i
    lst.append(tmp)
players = pd.concat(lst, axis=0, ignore_index=True)
players['player_id'] = players['Player_Link'].str.extract(r'/./(.*)\.html')
players.drop(columns='Player_Link', inplace=True)

In [106]:
players['MVP ranking'] = players['Awards'].str.extract(r'MVP-([0-9]*)').astype('Int64')

In [107]:
mvp_players = players[['player_id', 'MVP ranking', 'Season', 'Pos']].copy()
mvp_players.dropna(inplace=True)
mvp_players


,player_id,MVP ranking,Season,Pos
0,hardeja01,2,2019,PG
1,georgpa01,3,2019,SF
4,lillada01,6,2019,PG
5,duranke01,8,2019,SF
6,antetgi01,1,2019,PF
...,...,...,...,...
343,curryst01,9,2025,PG
345,jamesle01,6,2025,SF
346,mitchdo01,5,2025,SG
347,brunsja01,10,2025,PG


In [108]:
tbl_name = "AWARDS"
try:
    cnxn = mysql.connector.connect(**db_config)
    cursor = cnxn.cursor()
    cnxn.database = db_name

    cursor.execute(f"CREATE TABLE IF NOT EXISTS {tbl_name} ("
                   f"player_id VARCHAR(255) NOT NULL,"
                   f"Ranking INT NOT NULL,"
                   f"POS VARCHAR(255) NOT NULL,"
                   f"SEASON INT NOT NULL);")
    print(f"Table '{tbl_name}' created or already exists.")
    # Insert Dataframe into SQL Server:
    for index, row in mvp_players.iterrows():
        # print(f"inserting player {row['player_id']} year {row['Season']}")
        query = f"INSERT INTO {tbl_name} (player_id, Ranking, POS, SEASON) VALUES (%s, %s, %s, %s)"
        cursor.execute(query, (row['player_id'],row['MVP ranking'], row['Pos'],row['Season']))
    cnxn.commit()

except mysql.connector.Error as err:
    print(f"Error!!: {err}")
finally:
    if 'cnxn' in locals() and cnxn.is_connected():
        cursor.close()
        cnxn.close()
        print("Connection Closed.")

Table 'AWARDS' created or already exists.
Connection Closed.


In [120]:
top_players = players[['player_id', 'Rk', 'Age', 'Team', 'Pos', 'PTS', 'Season']].copy()
top_players

,player_id,Rk,Age,Team,Pos,PTS,Season
0,hardeja01,1,29,HOU,PG,2818,2019
1,georgpa01,2,28,OKC,SF,2159,2019
2,walkeke02,3,28,CHO,PG,2102,2019
3,bealbr01,4,25,WAS,SG,2099,2019
4,lillada01,5,28,POR,PG,2067,2019
...,...,...,...,...,...,...,...
379,banede01,46,26,MEM,SG,1327,2025
380,mobleev01,47,23,CLE,PF,1316,2025
381,powelno01,48,31,LAC,SG,1306,2025
382,bridgmi02,49,26,CHO,PF,1300,2025


In [123]:
tbl_name = "TOP_PLAYERS"
try:
    cnxn = mysql.connector.connect(**db_config)
    cursor = cnxn.cursor()
    cnxn.database = db_name

    cursor.execute(f"CREATE TABLE IF NOT EXISTS {tbl_name} ("
                   f"player_id VARCHAR(255) NOT NULL,"
                   f"Rk INT NOT NULL,"
                   f"AGE INT NOT NULL,"
                   f"team_id VARCHAR(20) NOT NULL,"
                   f"POS VARCHAR(20) NOT NULL,"
                   f"PTS INT NOT NULL,"
                   f"SEASON INT NOT NULL);")
    print(f"Table '{tbl_name}' created or already exists.")
    # Insert Dataframe into SQL Server:
    for index, row in top_players.iterrows():
        query = f"INSERT INTO {tbl_name} (player_id, Rk, AGE, team_id, POS, PTS, SEASON) VALUES (%s, %s, %s, %s, %s, %s, %s)"
        cursor.execute(query,(
            row['player_id'],
            row['Rk'],
            row['Age'],
            row['Team'],
            row['Pos'],
            row['PTS'],
            row['Season']))
    cnxn.commit()

except mysql.connector.Error as err:
    print(f"Error!!: {err}")
finally:
    if 'cnxn' in locals() and cnxn.is_connected():
        cursor.close()
        cnxn.close()
        print("Connection Closed.")

Table 'TOP_PLAYERS' created or already exists.
Connection Closed.


In [124]:
players_list = pd.read_csv(os.path.join('player stat', 'nba_players_stat.csv'))
players_list

,link,Name,Position,Shoots,Team,Born,Experience,Height_Weight
0,https://www.basketball-reference.com/players/l...,Kyle Lowry,Point Guard,Right,Philadelphia 76ers,"March 25,\n 1986\n (Age: 39-185d)\n...",19 years,"6-0,196lb (183cm, 88kg)"
1,https://www.basketball-reference.com/players/p...,Michael Porter Jr.,Small Forward,Right,Brooklyn Nets,"June 29,\n 1998\n (Age: 27-089d)\n\...",6 years,"6-10,218lb (208cm, 98kg)"
2,https://www.basketball-reference.com/players/v...,Nikola Vučević,Center,Right,Chicago Bulls,"October 24,\n 1990\n (Age: 34-337d)...",14 years,"6-10,260lb (208cm, 117kg)"
3,https://www.basketball-reference.com/players/h...,Montrezl Harrell,Center and Power Forward,Right,NaN,"January 26,\n 1994\n (Age: 31-243d)...",8 years,"6-7,240lb (201cm, 108kg)"
4,https://www.basketball-reference.com/players/b...,Desmond Bane,Shooting Guard,Right,Orlando Magic,"June 25,\n 1998\n (Age: 27-093d)\n\...",5 years,"6-5,215lb (196cm, 97kg)"
...,...,...,...,...,...,...,...,...
110,https://www.basketball-reference.com/players/m...,Bennedict Mathurin,Small Forward and Shooting Guard,Right,Indiana Pacers,"June 19,\n 2002\n (Age: 23-099d)\n\...",3 years,"6-6,210lb (198cm, 95kg)"
111,https://www.basketball-reference.com/players/p...,Norman Powell,Shooting Guard and Small Forward,Right,Miami Heat,"May 25,\n 1993\n (Age: 32-124d)\n\n...",10 years,"6-3,215lb (190cm, 97kg)"
112,https://www.basketball-reference.com/players/b...,Devin Booker,Shooting Guard and Point Guard,Right,Phoenix Suns,"October 30,\n 1996\n (Age: 28-331d)...",10 years,"6-6,206lb (198cm, 93kg)"
113,https://www.basketball-reference.com/players/w...,Victor Wembanyama,Center and Power Forward,Right,San Antonio Spurs,"January 4,\n 2004\n (Age: 21-265d)\...",2 years,"7-3,235lb (221cm, 106kg)"


In [125]:
# players_list['Age'] = players_list['Born'].str.extract(r'\(Age.*([0-9].)-').astype('Int64')
players_list['id'] = players_list['link'].str.extract(r'/./(.*)\.html')
# players_list['Experience'] = players_list['Experience'].str.extract(r'([0-9].)').astype('Int64')
players_list['Height'] = players_list['Height_Weight'].str.extract(r'([0-9]*)cm').astype('Int64')
players_list['Weight'] = players_list['Height_Weight'].str.extract(r'([0-9]*)kg').astype('Int64')
players_list.drop(columns=['Born', 'link', 'Height_Weight'], inplace=True)

In [137]:
players_list.drop_duplicates(inplace=True)

In [128]:
tbl_name = "PLAYERS_DETAIL"
try:
    cnxn = mysql.connector.connect(**db_config)
    cursor = cnxn.cursor()
    cnxn.database = db_name

    cursor.execute(f"CREATE TABLE IF NOT EXISTS {tbl_name} ("
                   f"player_id VARCHAR(255) NOT NULL PRIMARY KEY,"
                   f"NAME VARCHAR(255) NOT NULL,"
                   f"POS VARCHAR(255) NOT NULL,"
                   f"SHOOTS VARCHAR(10) NOT NULL,"
                   # f"TEAM VARCHAR(255),"
                   # f"EXPERIENCE INT,"
                   # f"AGE INT NOT NULL,"
                   f"HEIGHT INT NOT NULL,"
                   f"WEIGHT INT NOT NULL);")
    print(f"Table '{tbl_name}' created or already exists.")
    # Insert Dataframe into SQL Server:
    for index, row in players_list.iterrows():
        query = f"INSERT INTO {tbl_name} (player_id, NAME, POS, SHOOTS,  HEIGHT, WEIGHT) VALUES (%s, %s, %s, %s, %s, %s)"
        cursor.execute(query,(
            row['id'],
            row['Name'],
            row['Position'],
            row['Shoots'],
            # row['Team'] if pd.notna(row['Team']) else None,
            # row['Experience'] if pd.notna(row['Experience']) else None,
            # row['Age'],
            row['Height'],
            row['Weight']))
    cnxn.commit()

except mysql.connector.Error as err:
    print(f"Error!!: {err}")
finally:
    if 'cnxn' in locals() and cnxn.is_connected():
        cursor.close()
        cnxn.close()
        print("Connection Closed.")

Table 'PLAYERS_DETAIL' created or already exists.
Connection Closed.
